# Core: Applying Classifier for Mapping
Utilize already-trained model to create comprehensive LULC classifications from imagery, either downloaded or on-the-fly. Using chips and scoring models is not included. Not dependent on number of output categories.  
  
Date: 2020-01-14

Author: DC Team  

### Import statements

In [ ]:
# typical, comprehensive imports
import warnings
warnings.filterwarnings('ignore')
#
import os
import sys
import shapely
import cartopy
get_ipython().magic(u'matplotlib inline')
import matplotlib.pyplot as plt

import tensorflow as tf

import descarteslabs as dl
print (dl.places.find('illinois')) ## TEST

ULU_REPO = os.environ["ULU_REPO"]
if not ULU_REPO in sys.path:
    sys.path.append(ULU_REPO+'/utils')
    sys.path.append(ULU_REPO)
print (sys.path)

import util_vectors
import util_workflow
import util_mapping

## Preparation

### Set key variables

In [ ]:
# core
data_root='/data/phase_iv/'
place = 'johannesburg'
data_path=data_root+place+'/'

resolution = 5  # Lx:15 S2:10

# tiling
tile_resolution = resolution
tile_size = 256
tile_pad = 32

# ground truth source: aue, aue+osm, aue+osm2
# label_suffix = 'aue'

### Load and inspect study area

In [ ]:
# print place, place.title() # capitalized version of place name
place_title = place.title()
place_shapefile = data_path+place_title+"_studyAreaEPSG4326.shp"

# util_vectors.info_studyareas(data_path, place)

shape = util_vectors.load_shape(place_shapefile)
polygon = shape['geometry']['coordinates']
# print(polygon)
# pprint(shape)
place_bbox = shape['bbox']
# print(bbox)

# using Albers projection
lonlat_crs = cartopy.crs.PlateCarree()
clat, clon = (place_bbox[0]+place_bbox[2])/2.0, (place_bbox[1]+place_bbox[3])/2.0
print ("center co-ordinates", clat, clon)
albers = cartopy.crs.AlbersEqualArea(central_latitude=clat, central_longitude=clon)

# visualize Study Region
fig = plt.figure(figsize=(6,6))
ax = plt.subplot(projection=albers) # Specify projection of the map here
shp = shapely.geometry.shape(shape['geometry'])
ax.add_geometries([shp], lonlat_crs)
ax.set_extent((place_bbox[0], place_bbox[2], place_bbox[1], place_bbox[3]), crs=lonlat_crs)
ax.gridlines(crs=lonlat_crs)
plt.show()

### Generate tiles

In [ ]:
tiles = dl.raster.dltiles_from_shape(tile_resolution, tile_size, tile_pad, shape)
single_tile_id = 22
highlights = {single_tile_id:'green'}
util_vectors.draw_tiled_area(shape, tiles, albers, lonlat_crs, highlights=highlights)

### Set input stack and model parameters

In [ ]:
window = 17

model_id = '6cat_7city_EastAfrica'

### Load model
Includes loading custom loss function used during training

In [ ]:
# category_weights_filename = data_root+'models/'+model_id+'_category_weights.pkl'
# category_weights = pickle.load( open( category_weights_filename, "rb" ) )
# weights = list(zip(*category_weights.items())[1])

network_filename = data_root+'models/'+model_id+'.hd5'
network = tf.keras.models.load_model(
    network_filename,
    custom_objects={'loss': 'categorical_crossentropy'},
    compile=True
)
# network = K.load_model(network_filename, custom_objects={'loss': 'categorical-crossentropy'})
network.summary()

***

## Apply model: mapping
Apply model to imagery to classify all pixels in all non-empty tiles in all scenes.

In [ ]:
zfill=4
store_predictions=True
map_id=place+'_'+model_id+'_20191004'

In [ ]:
# johannesburg
scene_ids = ['sentinel-2:L1C:2019-10-04_35JNL_74_S2B_v1',
             'sentinel-2:L1C:2019-10-04_35JNM_52_S2B_v1',
             'sentinel-2:L1C:2019-10-04_35JPL_99_S2B_v1',
             'sentinel-2:L1C:2019-10-04_35JPM_99_S2B_v1']

# kampala
# scene_ids = ['sentinel-2:L1C:2019-12-31_36NVF_99_S2A_v1']

In [ ]:
util_mapping.map_scenes_simple(scene_ids, tiles, network, 
                               window=window, 
                               zfill=zfill, 
                               store_predictions=store_predictions, 
                               map_id=map_id)

### Combine model-generated tiles into mosaic
Optional step to create single, city-wide LULC raster for each scene mapped. Complete map is stored in same folder as constituent tiles.

In [ ]:
for scene_id in scene_ids:
    scene_id_short = scene_id[str.rfind(scene_id,':')+1:]
    if map_id is not None:
        scene_dir = data_root + 'scenes/' + map_id + '/' + scene_id_short
    else:
        scene_dir = data_root + 'scenes/' + scene_id_short
        
    path_template = scene_dir+'/'+scene_id_short+'_'+str(tile_resolution)+'m'+'_'+'p'+str(tile_pad)+'_'+\
            'tile'+'???????'[0:zfill]+'_'+'lulc'+'.tif'
    path_destination = scene_dir+'/'+scene_id_short+'_'+str(tile_resolution)+'m'+'_'+'p'+str(tile_pad)+'_'+\
            'complete'+'_'+'lulc'+'.tif'
    print(path_template)
    print(path_destination)
    !gdal_merge.py -n 255 -a_nodata 255 -o {path_destination} {path_template}


***